In [3]:
from pytqs import tqs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.ticker import PercentFormatter
from sklearn import neighbors
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
from sklearn.linear_model import LogisticRegression
import operator
import math
import pickle

# 聚类

已完成测试的用户

In [4]:
model_data = pd.read_csv('/home/tiger/archived-data/aeolus-data/20201222/15/34665529-定级测试信息利用-编辑框5-查询1.csv', encoding = 'gb18030')
# model_data.iloc[:, 1] = model_data.iloc[:, 1].fillna('unfilled')
model_data.head()

,user_id,finished,milestone_id,ttl_exe,A0_V_correct_rate,A1_V_correct_rate,A2_V_correct_rate,B1_V_correct_rate,B2_V_correct_rate,C1_V_correct_rate,...,B2_G_correct_rate,C1_G_correct_rate,C2_G_correct_rate,A0_L_correct_rate,A1_L_correct_rate,A2_L_correct_rate,B1_L_correct_rate,B2_L_correct_rate,C1_L_correct_rate,C2_L_correct_rate
0,6.153319e+14,1.0,21030.0,22,NaN,NaN,NaN,NaN,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.160730e+15,1.0,22010.0,26,NaN,1.0,1.0000,1.0,0.3333,0.0,...,0.3333,NaN,NaN,0.6667,0.3333,0.0,0.0,NaN,NaN,NaN
2,1.684042e+15,1.0,21030.0,19,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0000,NaN,NaN,NaN,NaN,NaN,NaN
3,4.111808e+15,1.0,21010.0,20,NaN,1.0,0.3333,0.0,NaN,NaN,...,NaN,NaN,NaN,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN
4,3.430103e+15,0.0,0.0,6,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
len(model_data)

401639

In [6]:
# dummy = pd.get_dummies(model_data['edu'])
# dummy.head()
model_data.head()

,user_id,finished,milestone_id,ttl_exe,A0_V_correct_rate,A1_V_correct_rate,A2_V_correct_rate,B1_V_correct_rate,B2_V_correct_rate,C1_V_correct_rate,...,B2_G_correct_rate,C1_G_correct_rate,C2_G_correct_rate,A0_L_correct_rate,A1_L_correct_rate,A2_L_correct_rate,B1_L_correct_rate,B2_L_correct_rate,C1_L_correct_rate,C2_L_correct_rate
0,6.153319e+14,1.0,21030.0,22,NaN,NaN,NaN,NaN,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.160730e+15,1.0,22010.0,26,NaN,1.0,1.0000,1.0,0.3333,0.0,...,0.3333,NaN,NaN,0.6667,0.3333,0.0,0.0,NaN,NaN,NaN
2,1.684042e+15,1.0,21030.0,19,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0000,NaN,NaN,NaN,NaN,NaN,NaN
3,4.111808e+15,1.0,21010.0,20,NaN,1.0,0.3333,0.0,NaN,NaN,...,NaN,NaN,NaN,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN
4,3.430103e+15,0.0,0.0,6,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# new_data = pd.concat([model_data, dummy], axis = 1)
new_data = model_data
new_data.columns

Index(['user_id', 'finished', 'milestone_id', 'ttl_exe', 'A0_V_correct_rate',
       'A1_V_correct_rate', 'A2_V_correct_rate', 'B1_V_correct_rate',
       'B2_V_correct_rate', 'C1_V_correct_rate', 'C2_V_correct_rate',
       'A0_G_correct_rate', 'A1_G_correct_rate', 'A2_G_correct_rate',
       'B1_G_correct_rate', 'B2_G_correct_rate', 'C1_G_correct_rate',
       'C2_G_correct_rate', 'A0_L_correct_rate', 'A1_L_correct_rate',
       'A2_L_correct_rate', 'B1_L_correct_rate', 'B2_L_correct_rate',
       'C1_L_correct_rate', 'C2_L_correct_rate'],
      dtype='object')

In [8]:
f_data = new_data[new_data.finished == 1.0]

f_data.iloc[:, np.r_[4:25]] = f_data.iloc[:, np.r_[4:25]].fillna(-1)
f_data.iloc[:, np.r_[4:25]].head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


,A0_V_correct_rate,A1_V_correct_rate,A2_V_correct_rate,B1_V_correct_rate,B2_V_correct_rate,C1_V_correct_rate,C2_V_correct_rate,A0_G_correct_rate,A1_G_correct_rate,A2_G_correct_rate,...,B2_G_correct_rate,C1_G_correct_rate,C2_G_correct_rate,A0_L_correct_rate,A1_L_correct_rate,A2_L_correct_rate,B1_L_correct_rate,B2_L_correct_rate,C1_L_correct_rate,C2_L_correct_rate
0,-1.0,-1.0,-1.0000,-1.0,0.0000,-1.0,-1.0,-1.0,-1.0000,-1.0000,...,-1.0000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,1.0,1.0000,1.0,0.3333,0.0,-1.0,-1.0,-1.0000,1.0000,...,0.3333,-1.0,-1.0,0.6667,0.3333,0.0,0.0,-1.0,-1.0,-1.0
2,1.0,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0000,-1.0000,...,-1.0000,-1.0,-1.0,1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,1.0,0.3333,0.0,-1.0000,-1.0,-1.0,-1.0,0.6667,0.6667,...,-1.0000,-1.0,-1.0,1.0000,0.0000,-1.0,-1.0,-1.0,-1.0,-1.0
6,1.0,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0000,-1.0000,...,-1.0000,-1.0,-1.0,0.5000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0


In [38]:
f_data.groupby('milestone_id').size()

milestone_id
21010.0    119605
21020.0     34129
21030.0     23080
22010.0     34147
22020.0     19189
22030.0     16129
23010.0     35453
23020.0      3010
23030.0     12477
24000.0      9666
25000.0       682
26000.0        80
dtype: int64

In [9]:
x = f_data.iloc[:, np.r_[4:25]]
y = f_data[['milestone_id']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=11)

KNN

In [52]:
knn = neighbors.KNeighborsClassifier(algorithm='kd_tree',  n_neighbors = 15)
knn.fit(x_train, y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=15)

In [106]:
start = time.time()
y_pred = knn.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
end = time.time()
print((end - start)/len(y_pred))

Accuracy: 0.6868570298074387
0.008572979114578255


Linear Regression

In [10]:
lm = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(x_train, y_train)

start = time.time()
y_pred = lm.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
end = time.time()
print((end - start)/len(y_pred))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter)

Accuracy: 0.5731296386586489
4.587985110622592e-07


In [11]:
(0.57-0.65)/0.65

-0.12307692307692318

In [12]:
# save parameter
coef = pd.DataFrame(lm.coef_, columns = x_test.columns)
intercept = pd.DataFrame(lm.intercept_, columns = ['intercept'])

parameter = pd.concat([intercept, coef], axis=1)
parameter.to_csv(r'./logistic_parameter.csv', encoding='utf_8_sig')

In [36]:
parameter

,intercept,A0_V_correct_rate,A1_V_correct_rate,A2_V_correct_rate,B1_V_correct_rate,B2_V_correct_rate,C1_V_correct_rate,C2_V_correct_rate,A0_G_correct_rate,A1_G_correct_rate,...,B2_G_correct_rate,C1_G_correct_rate,C2_G_correct_rate,A0_L_correct_rate,A1_L_correct_rate,A2_L_correct_rate,B1_L_correct_rate,B2_L_correct_rate,C1_L_correct_rate,C2_L_correct_rate
0,-2.379669,1.041146,6.224346,-4.814325,1.008713,-2.253395,-0.612394,0.672493,2.219963,-1.395255,...,-2.255612,2.219963,2.219963,0.797576,-1.514920,-1.698722,-0.119777,-2.991462,0.935041,-0.399557
1,-1.299363,-0.457906,-1.256065,0.899716,-0.384003,-0.366170,-0.734272,-0.365238,1.215565,0.765225,...,-0.765987,1.215565,1.215565,0.409693,0.290069,0.387594,-0.528294,-0.502272,-0.160111,-0.232479
2,-1.116740,-0.587887,-1.241944,0.207058,0.014960,0.312462,-0.490678,-0.293371,1.080035,0.699014,...,0.200767,1.080035,1.080035,0.246281,0.418438,0.308738,-0.798631,-1.355159,0.028158,-0.392157
3,-1.030193,-0.701642,-1.223614,0.099308,0.077806,0.276616,-0.527862,-0.605608,1.013862,0.036679,...,-0.081798,1.013862,1.013862,-0.118643,0.889311,0.254349,-0.225214,-0.142966,-0.943343,-0.273285
4,-3.985036,-0.511934,-0.630273,0.091545,-0.238578,0.455208,-0.268039,-0.378378,0.007886,0.131574,...,0.241026,0.007886,0.007886,-0.333348,0.287053,0.290867,0.332072,0.197682,-0.954833,-0.420906
5,-4.327814,-0.544299,-0.653985,0.313334,-0.393240,0.424767,-0.376588,-0.253581,0.002087,-0.256719,...,0.644487,0.002087,0.002087,-0.472257,0.009580,0.800518,0.085956,0.131833,-0.791064,-0.298661
6,-4.237244,-0.360251,-0.521110,0.140490,-0.203283,0.913694,-0.029982,-0.687955,0.066475,-0.434624,...,0.744684,0.066475,0.066475,-0.780652,-0.052468,1.389923,-0.472035,1.069421,-0.840102,-0.738197
7,-5.579797,-0.244717,-0.192209,0.062766,0.460714,0.626682,-0.385543,-0.029926,-0.000096,-0.144810,...,0.096386,-0.000096,-0.000096,-0.939963,-0.162999,0.486488,0.608878,1.696970,-0.621932,-0.514268
8,-5.185507,-0.022609,-0.181695,0.290217,0.155996,1.697022,0.091728,0.153063,-0.002232,-0.381016,...,0.805774,-0.002232,-0.002232,-0.951884,-0.410078,0.443700,1.348809,2.057487,-0.076943,0.085232
9,-5.872471,-0.278064,-0.015998,-0.365968,0.334570,2.062037,0.678301,-0.270161,-0.000024,-0.124506,...,0.577308,-0.000024,-0.000024,-1.314799,-0.740413,-0.673515,0.356332,1.751534,-0.054108,0.048426


In [80]:
# lm.predict(x_test.head(1))

In [13]:
# save test data
# x_test.to_csv(r'./x_test.csv', encoding='utf_8_sig')

In [14]:
# save vallidate data

x_test['new_index'] = range(0, len(x_test))
x_test.index = x_test['new_index'] 

y_pred = pd.DataFrame(y_pred)
y_pred.columns =['new_mile_stone'] 

validate_data = pd.concat([x_test, y_pred], axis=1)
validate_data.to_csv(r'./logistic_model_validate.csv', encoding='utf_8_sig')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
unique, counts = np.unique(y_test, return_counts=True)
dict(zip(unique, counts))

{21010.0: 35823,
 21020.0: 10319,
 21030.0: 6958,
 22010.0: 10170,
 22020.0: 5755,
 22030.0: 4808,
 23010.0: 10695,
 23020.0: 902,
 23030.0: 3718,
 24000.0: 2923,
 25000.0: 200,
 26000.0: 24}

In [18]:
# 21010/len(y_test)

未完成测试的用户

In [19]:
uf_data = model_data[model_data.finished == 0.0]
uf_data[uf_data.columns[4:25]] = uf_data[uf_data.columns[4:25]].fillna(-1)

x_train = f_data[f_data.columns[4:25]]
y_train = f_data[['milestone_id']]

x_test = uf_data[uf_data.columns[4:25]]
y_test = uf_data[['milestone_id']]


# knn = neighbors.KNeighborsClassifier(algorithm='kd_tree',  n_neighbors = 15)
# knn.fit(x_train, y_train)

y_pred = lm.predict(x_test)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [20]:
y_pred

array([21010., 21010., 21010., ..., 22010., 21010., 21010.])

In [21]:
len(y_pred)

93991

In [22]:
unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{21010.0: 69682,
 21020.0: 1507,
 21030.0: 1852,
 22010.0: 8613,
 22020.0: 118,
 22030.0: 165,
 23010.0: 10441,
 23020.0: 30,
 23030.0: 744,
 24000.0: 834,
 25000.0: 5}

In [23]:
uf_data.head()

,user_id,finished,milestone_id,ttl_exe,A0_V_correct_rate,A1_V_correct_rate,A2_V_correct_rate,B1_V_correct_rate,B2_V_correct_rate,C1_V_correct_rate,...,B2_G_correct_rate,C1_G_correct_rate,C2_G_correct_rate,A0_L_correct_rate,A1_L_correct_rate,A2_L_correct_rate,B1_L_correct_rate,B2_L_correct_rate,C1_L_correct_rate,C2_L_correct_rate
4,3.430103e+15,0.0,0.0,6,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,3.043080e+15,0.0,0.0,4,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
15,3.254192e+15,0.0,0.0,7,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
21,2.849554e+15,0.0,0.0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
26,9.847973e+14,0.0,0.0,9,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [24]:
df1 = uf_data[["user_id", "ttl_exe"]]
df1['new_index'] = range(0, len(df1))
df1.index = df1['new_index'] 
df1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,ttl_exe,new_index
new_index,,,
0,3.430103e+15,6,0
1,3.043080e+15,4,1
2,3.254192e+15,7,2
3,2.849554e+15,1,3
4,9.847973e+14,9,4
...,...,...,...
93986,3.078259e+15,1,93986
93987,2.198637e+15,7,93987
93988,3.465273e+15,15,93988


In [25]:
df2 = pd.DataFrame(y_pred)
df2.columns =['new_mile_stone'] 
df2

,new_mile_stone
0,21010.0
1,21010.0
2,21010.0
3,21010.0
4,21010.0
...,...
93986,21010.0
93987,21010.0
93988,22010.0
93989,21010.0


In [26]:
new_df = pd.concat([df1, df2], axis=1)
new_df.head()

,user_id,ttl_exe,new_index,new_mile_stone
0,3.430103e+15,6,0,21010.0
1,3.043080e+15,4,1,21010.0
2,3.254192e+15,7,2,21010.0
3,2.849554e+15,1,3,21010.0
4,9.847973e+14,9,4,21010.0


# 新等级和默认等级对比

In [29]:
fdk_data = pd.read_csv('/home/tiger/archived-data/aeolus-data/20201117/14/31687312-用户行为日志分析-挽留弹窗用户分析-查询25.csv', encoding = 'gb18030')
# fdk_data.head()

fdk_user = fdk_data[["user_id", "feedback_answer_new", "is_default_level", "lesson_level", "role"]]
fdk_user = fdk_user.loc[(fdk_user["feedback_answer_new"] == "401") | (fdk_user["feedback_answer_new"] == "402")]
fdk_user.head()

,user_id,feedback_answer_new,is_default_level,lesson_level,role
3,4432853049825111,401,1.0,A1中,上班族
4,3025484090719454,401,0.0,A1初,中小学生
7,4362485978366104,401,0.0,A1初,中小学生
12,2937527789947293,401,1.0,A1中,NaN
16,3078254632895918,402,0.0,A2中,大学生


In [30]:
def_user_all = fdk_user[(fdk_user.is_default_level == 1.0)]
def_user_all.loc[(def_user_all['feedback_answer_new'] == "401"), 'feedback'] = '太难了'
def_user_all.loc[(def_user_all['feedback_answer_new'] == "402"), 'feedback'] = '太简单了'
def_user_all.groupby(['feedback']).size()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


feedback
太简单了     588
太难了     1178
dtype: int64

In [31]:
new_df.head()

,user_id,ttl_exe,new_index,new_mile_stone
0,3.430103e+15,6,0,21010.0
1,3.043080e+15,4,1,21010.0
2,3.254192e+15,7,2,21010.0
3,2.849554e+15,1,3,21010.0
4,9.847973e+14,9,4,21010.0


In [32]:
result = pd.merge(def_user_all, new_df, 'right' , on ='user_id')
result.head()

,user_id,feedback_answer_new,is_default_level,lesson_level,role,feedback,ttl_exe,new_index,new_mile_stone
0,3430103318019351,NaN,NaN,NaN,NaN,NaN,6,0,21010.0
1,3043079722116622,NaN,NaN,NaN,NaN,NaN,4,1,21010.0
2,3254192456604205,NaN,NaN,NaN,NaN,NaN,7,2,21010.0
3,2849554127664551,NaN,NaN,NaN,NaN,NaN,1,3,21010.0
4,984797323210264,NaN,NaN,NaN,NaN,NaN,9,4,21010.0


In [33]:
len(new_df)

93991

In [34]:
len(result)

94006

In [35]:
result.groupby(['new_mile_stone' ,'feedback']).size()

new_mile_stone  feedback
21010.0         太简单了         99
                太难了         764
21020.0         太简单了          3
                太难了           1
21030.0         太简单了          5
                太难了           3
22010.0         太简单了         20
                太难了          14
23010.0         太简单了         46
                太难了          11
23030.0         太简单了          8
dtype: int64

In [86]:
(159+711)/( 59817)

0.014544360298911681

In [87]:
(159)/(59817)

0.0026581072270424797

In [88]:
(711)/(59817)

0.011886253071869201

In [89]:
(59817 - 711-159)/(59817)

0.9854556397010883

In [228]:
result.to_csv(r'./mile_stone_result.csv', encoding='utf_8_sig')